## Data

In [ ]:
examples = [
  "statement positive test this is a lovely test .",
  "statement negative test this is a horrible test .",
  "statement neutral test this is a test .",
]
batch_of_tokens = list(map(lambda sentence:sentence.split(),examples))

inputs = []
targets = []
for tokens in batch_of_tokens:
  for index in range(1,len(tokens)):
    context = ' '.join(tokens[:index])
    inputs.append(context)
    targets.append(tokens[index])

## Encoding

In [ ]:
from ffast import load

tokeniser = load('poincare')

In [ ]:
from numpy import array, zeros

def one_hot(id, max_len):
  vector = zeros(max_len)
  vector[id] = 1 
  return vector

max_length = len(tokeniser)
encode_inputs = lambda sentence: tokeniser.encode(sentence).vector
encode_targets = lambda word: one_hot(tokeniser.encode(word).ids[0], max_length)

In [ ]:
X = array(list(map(encode_inputs,inputs)))
T = array(list(map(encode_targets,targets)))

## Train Model

In [ ]:
from windML import RecurrentELM

model = RecurrentELM(tokeniser.size, len(tokeniser))
model.fit(X,T)

## Save Model

In [ ]:
model.save("my_nlg")
model = RecurrentELM(tokeniser.size, len(tokeniser), load_path="my_nlg")

## Use Model

In [ ]:
prompt = "statement positive test"
generated_ids = list(model.generate(
    prompt_token_ids=tokeniser.encode(prompt).ids,
    token_id_vectoriser=lambda token_ids:tokeniser.decode(token_ids).vector
))
str(tokeniser.decode(generated_ids))